In [1]:
!pip3 install spacy -q
!python3 -m spacy download en -q
!python3 -m spacy download en_core_news_md -q

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

✘ No compatible package found for 'en_core_news_md' (spaCy v3.2.4)



### importer les differentes librairies

In [2]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from urllib.parse import urlparse

### lire nos données et transformer les colonnes majuscules en minuscules

In [3]:
df = pd.read_csv('/home/jovyan/Projects/projet_perso/train.csv')
df.columns = df.columns.str.lower()
df.sample(10)

,labels,text,text_tag
4116,5,The world food demand is going to double somet...,poverty
2575,3,Says he earned the highest possible credit rat...,"government-efficiency,job-accomplishments,mess..."
105,5,Sen. Obama has always had a 100 percent procho...,abortion
4647,0,"In 2009, his first year as mayor, Julin Castro...","campaign-finance,candidates-biography,ethics"
8354,0,The most popular name is no longer John or Ste...,"diversity,families,immigration"
4496,5,Governor Palin is the most popular governor in...,candidates-biography
595,0,Kendrick Meek was #1 in Florida in taking toba...,"campaign-finance,candidates-biography"
3905,2,Says Michael Bennet votes for President Obamas...,"bipartisanship,voting-record"
1514,2,Women in Barack Obamas White House are earning...,"jobs,women"
2055,3,Says House Republicans rejected a straightforw...,"ethics,federal-budget,voting-record,workers"


### Filtrer nos données par rapport aux labels qui sont différents de 4 (4 = Iconnu)

In [4]:
df = df[df["labels"] != 4]

### supprimer les valeurs manquantes

In [5]:
df.isnull().any()
df["text_tag"].isnull().sum()
df=df.dropna()
df.shape

(9399, 3)

### transformer nos labels multivariés en bivariés ( 1 = Fake news, 0 = vrai info)

In [6]:
df["labels"] =df["labels"].apply(lambda x : 0 if x == 3 or x == 5
                       else 1)
df["labels"].value_counts()

1    5761
0    3638
Name: labels, dtype: int64

In [7]:
df.sample(10)

,labels,text,text_tag
7917,0,"I used tax cuts to help create over 80,000 job...",taxes
7734,1,Will (HB2)threaten federal funding for public ...,"education,gays-and-lesbians,government-regulat..."
1732,1,Cory Gardner is sponsoring a bill to ban all a...,"abortion,message-machine-2014,women"
7511,1,After initially agreeing with Republicans on a...,"abortion,public-health"
3142,1,"Says these Jersey Shore folks, theyre from New...",pop-culture
9640,0,We have cut our deficits by almost three-quart...,"deficit,economy"
9827,1,Betty Sutton gutted Medicare by $500 billion w...,"health-care,medicare,voting-record"
2623,1,Says Hillary Clinton has been in office and in...,candidates-biography
4620,1,There are still more people uninsured today th...,health-care
1778,1,"Since President Obama came into office, anothe...","diversity,poverty"


### transformer tous les tag_text majuscules en minuscules

In [8]:
df["text_tag"] = df["text_tag"].fillna("").str.lower()

In [9]:
tags = df.text_tag.unique()
tags

array(['abortion', 'energy,history,job-accomplishments', 'foreign-policy',
       ..., 'animals,elections', 'retirement,social-security',
       'florida,foreign-policy'], dtype=object)

### rajouter nos tag_text au text

In [10]:
df["text"] = df["text"] + " " + df["text_tag"].str.replace(",", " ")
df["text"][0]

'Says the Annies List political group supports third-trimester abortions on demand. abortion'

supprimer la colonnes text_tag

In [11]:
df = df.drop(columns = "text_tag")
df.head()

,labels,text
0,1,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...
2,0,"Hillary Clinton agrees with John McCain ""by vo..."
3,1,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...


In [12]:
df["text"][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. energy history job-accomplishments'

### Stemming du text

In [16]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import SnowballStemmer

en = SnowballStemmer('english')
df["text_stem"] = df["text"].map(lambda s: [en.stem(word) for word in s.split(" ")])

df[["text", "text_stem"]].iloc[1].tolist()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. energy history job-accomplishments',
 ['when',
  'did',
  'the',
  'declin',
  'of',
  'coal',
  'start?',
  'it',
  'start',
  'when',
  'natur',
  'gas',
  'took',
  'off',
  'that',
  'start',
  'to',
  'begin',
  'in',
  '(presid',
  'georg',
  'w.)',
  'bush',
  'administration.',
  'energi',
  'histori',
  'job-accomplish']]

In [17]:
all_text = " ".join(df["text_stem"].map(lambda l: " ".join(l)).tolist())
len(set(all_text.split(" ")))

15529

In [18]:
df.head()

,labels,text,text_stem
0,1,Says the Annies List political group supports ...,"[say, the, anni, list, polit, group, support, ..."
1,1,When did the decline of coal start? It started...,"[when, did, the, declin, of, coal, start?, it,..."
2,0,"Hillary Clinton agrees with John McCain ""by vo...","[hillari, clinton, agre, with, john, mccain, ""..."
3,1,Health care reform legislation is likely to ma...,"[health, care, reform, legisl, is, like, to, m..."
4,1,The economic turnaround started at the end of ...,"[the, econom, turnaround, start, at, the, end,..."


In [19]:
df["text_stem"][1]

['when',
 'did',
 'the',
 'declin',
 'of',
 'coal',
 'start?',
 'it',
 'start',
 'when',
 'natur',
 'gas',
 'took',
 'off',
 'that',
 'start',
 'to',
 'begin',
 'in',
 '(presid',
 'georg',
 'w.)',
 'bush',
 'administration.',
 'energi',
 'histori',
 'job-accomplish']

In [20]:
import pickle
df.to_pickle('df_st_s.pk')

In [21]:
en.stem("generously")

'generous'

In [22]:
from nltk.stem import PorterStemmer

In [23]:
ps = PorterStemmer()
ps.stem("generously")

'gener'